In [31]:
import pdfplumberimport osimport jsonimport pandas as pdimport reimport timefrom openai import OpenAI# Configuração da API Keyclient = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))if not os.getenv("OPENAI_API_KEY"):    raise ValueError("OPENAI_API_KEY não encontrada. Configure a variável de ambiente.")# Base de pastasimport sysfrom pathlib import Pathsys.path.append(str(Path().absolute().parent.parent / "src"))from utils.config import ITR_2024_PATHbase_folder = str(ITR_2024_PATH)# Regexes avançados para pré-filtroregexes = [    r"CDI\s*\+\s*\d+,\d+%",    r"TJLP\s*\+\s*\d+,\d+%",    r"SOFR\s*\+\s*\d+,\d+%",    r"\d+,\d+%\s*ao\s+ano",    r"\d+,\d+%\s*a\.a\.",    r"\d+,\d+%\s*aa",    r"\d+%.*CDI",    r"CDI.*\d+,\d+%",    r"juros.*\d+,\d+%",    r"remuneração.*\d+,\d+%",    r"custo.*\d+,\d+%",    r"\d+,\d+%\s*a\.a",    r"\d+,\d+%\s*aa",    r"\d+,\d+%\s*ao\s+ano\s*\+\s*variação\s+cambial",    r"variação\s+cambial.*\d+,\d+%"]def texto_tem_indicio_financiamento(texto):    for rex in regexes:        if re.search(rex, texto, flags=re.IGNORECASE):            return True    return Falsedef processar_pasta(pasta_path):    dados_extraidos = []    nome_pasta = os.path.basename(pasta_path)    csv_path = os.path.join(pasta_path, f"LLM_{nome_pasta}.csv")    if os.path.exists(csv_path):        print(f"⚠️ CSV já existe para {nome_pasta}, pulando...")        return    for file in os.listdir(pasta_path):        if not file.lower().endswith(".pdf"):            continue        empresa_nome = os.path.splitext(file)[0]        path = os.path.join(pasta_path, file)        print(f"\n📂 Lendo: {file} da pasta {nome_pasta}")        try:            with pdfplumber.open(path) as pdf:                texto_total = ""                for page in pdf.pages:                    texto_total += page.extract_text() or ""        except Exception as e:            print(f"❌ Erro ao abrir PDF {file}: {e}")            continue        if not texto_total.strip():            continue        # PRÉ-FILTRO        if not texto_tem_indicio_financiamento(texto_total):            print(f"🚫 Nenhum indicio de financiamento em {file}, ignorando.")            continue        # DIVIDE EM CHUNKS DE 60.000        chunk_size = 60000        chunks = [texto_total[i:i+chunk_size] for i in range(0, len(texto_total), chunk_size)]        for idx, chunk_text in enumerate(chunks, start=1):            prompt = f"""Você receberá o texto extraído das demonstrações financeiras de uma empresa.Identifique TODOS os financiamentos / empréstimos, mesmo que o nome do contrato esteja em uma tabela e a taxa, prazo ou valor estejam em outro local do texto. Associe as informações mesmo se estiverem separadas.Se encontrar dívidas denominadas em moeda estrangeira (ex: dólares) mas com contrato de swap ou instrumento derivativo para reais com taxa CDI + spread, considere o custo efetivo como a taxa pós-swap.Além disso, identifique o CNPJ da empresa, se houver, e retorne no campo "cnpj". Caso não encontre, deixe "".Para cada financiamento, extraia:- "item": o nome do financiamento- "valor": o saldo ou valor do financiamento- "taxa": custo ou taxa associada- "kd_bruto": valor numérico anual em %, EX: para "CDI + 1,15%" com CDI 10,88% a.a., informe "12.03"- "kd_liquido": calcule kd_bruto * (1 - 0.34) e informe, EX: "7.94"- "prazo": vencimento ou prazo- "descricao": breve contexto ou trecho encontrado- "cnpj": o CNPJ da empresa encontrado no textoRetorne apenas um JSON array assim:[    {{        "item": "...",        "valor": "...",        "taxa": "...",        "kd_bruto": "...",        "kd_liquido": "...",        "prazo": "...",        "descricao": "...",        "cnpj": "..."    }},    ...]Sempre use ponto (.) como separador decimal e duas casas.  Se não encontrar nada, retorne apenas [].Texto:{chunk_text}"""            inicio_tempo = time.time()            response = client.chat.completions.create(                model="gpt-4o-mini",                messages=[{"role": "user", "content": prompt}]            )            fim_tempo = time.time()            duracao = round(fim_tempo - inicio_tempo, 2)            resposta = response.choices[0].message.content            print(f"📝 Resposta GPT ({duracao}s) chunk {idx}: {resposta[:300]}...\n")            try:                json_data = json.loads(re.search(r"\[.*\]", resposta, re.DOTALL).group())            except Exception as e:                print(f"❌ Erro ao parsear JSON chunk {idx} para {file}: {e}")                json_data = []            if json_data:                for item in json_data:                    dados_extraidos.append({                        "arquivo_pdf": file,                        "empresa": empresa_nome,                        "chunk_num": idx,                        "item": item.get("item", ""),                        "valor": item.get("valor", ""),                        "taxa": item.get("taxa", ""),                        "kd_bruto": item.get("kd_bruto", ""),                        "kd_liquido": item.get("kd_liquido", ""),                        "prazo": item.get("prazo", ""),                        "descricao": item.get("descricao", ""),                        "cnpj": item.get("cnpj", ""),                        "tempo_execucao_segundos": duracao                    })                time.sleep(1.5)            else:                print(f"⚠️ Nenhum financiamento identificado no chunk {idx} de {file}")    if dados_extraidos:        df = pd.DataFrame(dados_extraidos)        df = df.sort_values(by=["arquivo_pdf", "chunk_num", "item"])        df.to_csv(csv_path, index=False, encoding="utf-8-sig")        print(f"✅ CSV salvo em '{csv_path}'")    else:        print(f"⚠️ Nenhum dado relevante encontrado na pasta '{nome_pasta}'")# def main():#     for entry in os.scandir(base_folder):#         if entry.is_dir():#             print(f"\n=== Processando pasta: {entry.name} ===")#             processar_pasta(entry.path)def main():    for entry in os.scandir(base_folder):        if entry.is_dir():            # Testar só a pasta Arezzo (ajuste o nome conforme o da sua pasta)            if entry.name.lower() == "arezzo":                print(f"\n=== Processando pasta: {entry.name} ===")                processar_pasta(entry.path)if __name__ == "__main__":    main()



=== Processando pasta: AREZZO ===

📂 Lendo: AREZZO_1T24.pdf da pasta AREZZO
📝 Resposta GPT (6.45s) chunk 1: ```json
[
    {
        "item": "Empréstimos e Financiamentos (Passivo Circulante)",
        "valor": "385015",
        "taxa": "",
        "kd_bruto": "",
        "kd_liquido": "",
        "prazo": "",
        "descricao": "Valor total de empréstimos e financiamentos a curto prazo.",
        "cnpj"...

📝 Resposta GPT (7.11s) chunk 2: ```json
[
    {
        "item": "Finame",
        "valor": "42.00",
        "taxa": "6.00%",
        "kd_bruto": "6.00",
        "kd_liquido": "3.96",
        "prazo": "outubro de 2024",
        "descricao": "Empréstimo com taxa de 6% ao ano, com parcelas mensais.",
        "cnpj": " "
    },
    {
...

📝 Resposta GPT (3.24s) chunk 3: ```json
[
    {
        "item": "Financiamento Arezzo",
        "valor": "5.419.000",
        "taxa": "CDI + 1.15%",
        "kd_bruto": "12.03",
        "kd_liquido": "7.94",
        "prazo": "31/03/2024",
        "d

In [1]:
import osimport pandas as pd# Base de pastas onde estão os CSVsimport sysfrom pathlib import Pathsys.path.append(str(Path().absolute().parent.parent / "src"))from utils.config import ITR_2024_PATHbase_folder = str(ITR_2024_PATH)# Pasta onde vai salvar o arquivo finalimport sysfrom pathlib import Pathsys.path.append(str(Path().absolute().parent.parent / "src"))from utils.config import CONSOLIDATED_PATHoutput_folder = str(CONSOLIDATED_PATH)output_file = os.path.join(output_folder, "Validacao_IA.xlsx")# Lista para guardar todos os DataFramesall_dfs = []# Percorre recursivamentefor root, dirs, files in os.walk(base_folder):    for file in files:        if file.startswith("LLM_") and file.endswith(".csv"):            csv_path = os.path.join(root, file)            print(f"📥 Lendo {csv_path}")            try:                df = pd.read_csv(csv_path)                all_dfs.append(df)                                # Adiciona linha em branco                empty_row = pd.DataFrame([[""] * len(df.columns)], columns=df.columns)                all_dfs.append(empty_row)            except Exception as e:                print(f"⚠️ Erro ao ler {csv_path}: {e}")if all_dfs:    # Concatena todos    final_df = pd.concat(all_dfs, ignore_index=True)    final_df.to_excel(output_file, index=False, engine='openpyxl')    print(f"\n✅ Arquivo Excel gerado em '{output_file}'")else:    print("\n🚫 Nenhum CSV encontrado para consolidar.")

📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/PETRORIO/LLM_PETRORIO.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/GERDAU/LLM_GERDAU.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/DEXCO/LLM_DEXCO.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/DEXXOS_PART/LLM_DEXXOS_PART.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/AFLUENTE_T/LLM_AFLUENTE_T.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/ALLIAR/LLM_ALLIAR.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/GRUPO_MATEUS/LLM_GRUPO_MATEUS.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMESTRAIS_2024/ESPAÇOLASER/LLM_ESPAÇOLASER.csv
📥 Lendo /Users/thiagomoncinhatto/Documents/02_MBA_Financas/TCC/DFs/DFS_TRIMEST

In [29]:
# import os# import sysfrom pathlib import Pathsys.path.append(str(Path().absolute().parent.parent / "src"))from utils.config import ITR_2024_PATHbase_folder = str(ITR_2024_PATH)# def excluir_csvs():#     total_removidos = 0#     for entry in os.scandir(base_folder):#         if entry.is_dir():#             pasta = entry.path#             for file in os.listdir(pasta):#                 if file.startswith("LLM_") and file.endswith(".csv"):#                     caminho = os.path.join(pasta, file)#                     try:#                         os.remove(caminho)#                         print(f"🗑️ Removido: {caminho}")#                         total_removidos += 1#                     except Exception as e:#                         print(f"❌ Erro ao remover {caminho}: {e}")#     print(f"\n✅ Total de arquivos CSV removidos: {total_removidos}")# if __name__ == "__main__":#     excluir_csvs()
